# Project Part 2

[![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/sgeinitz/CS39AA-project/blob/main/project_part1.ipynb)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sgeinitz/CS39AA-project/blob/main/project_part1.ipynb)

## 1. Introduction/Background

For project part three  I will be using a couple different song datasets that contain the song title, the artist,  the genre(s) of the song, and I will add some different song attributes to each song. I will also be using my own dataset with my personal spotify data of liked songs, this time also using the same song attributes. This personal dataset will have the same parameters along with a parameter that will say 1 or 0 if I like the song or do not.Initially the liked songs dataset will have a like value of 1 and the other dataset i have not seen will all have like values of 0. Instead of changing those values i am going to add another column called "like prediction' that will change based on if the song genre and attributes match my own tase in music. All in all, I am using song data to predict if I would like a song based on my own liked songs

## 2. Exploratory Data Analysis

for part 3 of my project I will be using this dataset: https://www.kaggle.com/datasets/byomokeshsenapati/spotify-song-attributes

I wanted to use a larger and more refined dataset for this but in the end it was going to take too much time to generate genres and song attributes for a dataset with 100,000 entries. If the script i wrote to format my own dataset were to be used on the dataset of over 100,000 entries it would take over 9 hours to run through. This dataset has over 10,000 entries and already has genre and song attributes so it will do. 


In [43]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, classification_report
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout


Here is the dataset found from kaggle linked above. Here i read it into a file and format it so the only values displayed are trackName, artistName, and genre. Then I add a like column populated with all zeros since these are the songs i will be predicting and they start with not liked. 

In [44]:
df = pd.read_csv('https://raw.githubusercontent.com/CooperDavis00/CS39AA-Project/main/Spotify_Song_Attributes.csv')
df = df.drop(['msPlayed','time_signature', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms'], axis=1)
df['like'] = 0
df['predicted like'] = 0
df = df.dropna(subset=['genre'])

df = pd.get_dummies(df, columns=['key', 'mode'])

df.head()

,trackName,artistName,genre,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,...,key_4.0,key_5.0,key_6.0,key_7.0,key_8.0,key_9.0,key_10.0,key_11.0,mode_0.0,mode_1.0
1,"""In The Hall Of The Mountain King"" from Peer G...",London Symphony Orchestra,british orchestra,0.475,0.130,-17.719,0.0510,0.9160,0.956000,0.1010,...,False,False,False,True,False,False,False,False,False,True
2,#BrooklynBloodPop!,SyKo,glitchcore,0.691,0.814,-3.788,0.1170,0.0164,0.000000,0.3660,...,False,False,False,False,False,False,False,False,True,False
3,$10,Good Morning,experimental pop,0.624,0.596,-9.804,0.0314,0.4750,0.203000,0.1190,...,True,False,False,False,False,False,False,False,False,True
4,(I Just) Died In Your Arms,Cutting Crew,album rock,0.625,0.726,-11.402,0.0444,0.0158,0.000169,0.0625,...,False,False,False,False,False,False,False,True,True,False
5,(L)only Child,salem ilese,alt z,0.645,0.611,-5.925,0.1370,0.2900,0.000021,0.2370,...,False,False,False,False,True,False,False,False,True,False


Here is my dataset generated from my own spotify playlist with the same columns and i added a like column populated with ones. 


In [45]:
df1 = pd.read_csv('https://raw.githubusercontent.com/CooperDavis00/CS39AA-Project/main/liked_w_features.csv')
df1['like'] = 1
df1['predicted like'] = 0
df1 = df1.drop('Track URI', axis = 1)

df1 = pd.get_dummies(df1, columns=['key', 'mode'])


df1.head()

,Track Name,Artist Name(s),genre,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,...,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11,mode_0,mode_1
0,Gamesofluck,Parcels,"aussietronica, indie soul",0.659,0.757,-5.275,0.0380,0.12800,0.48500,0.0859,...,False,False,False,False,False,True,False,False,True,False
1,So Much In Love - Armin van Buuren Remix,"D.O.D, Armin van Buuren","electro house, house, pop dance, uk dance, dut...",0.672,0.769,-5.887,0.0788,0.04770,0.00488,0.4390,...,False,False,False,False,False,False,False,False,False,True
2,Envious - Rezident Remix,"Aluna, Rezident","dance pop, german house, melodic house",0.659,0.710,-7.476,0.0341,0.14300,0.17300,0.0900,...,False,False,False,False,False,False,False,False,True,False
3,Change Your Mind,Interupt,bassline,0.677,0.864,-5.466,0.0689,0.00571,0.00478,0.3560,...,False,False,False,False,False,False,False,False,True,False
4,Isolate,D.O.D,"electro house, house, pop dance, uk dance",0.662,0.884,-4.669,0.0500,0.20900,0.16200,0.0980,...,True,False,False,False,False,False,False,False,True,False


The column names do not match up so I need to make sure each column matches. 

In [47]:
# Standardizing column names

df1.rename(columns={'Track Name': 'trackName', 'Artist Name(s)': 'artistName', 'Genre': 'genre'}, inplace=True)


print("Columns in df1:", df1.columns)
print("Columns in df:", df.columns)


Columns in df1: Index(['trackName', 'artistName', 'genre', 'danceability', 'energy',
       'loudness', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'like', 'predicted like', 'key_0',
       'key_1', 'key_2', 'key_3', 'key_4', 'key_5', 'key_6', 'key_7', 'key_8',
       'key_9', 'key_10', 'key_11', 'mode_0', 'mode_1'],
      dtype='object')
Columns in df: Index(['trackName', 'artistName', 'genre', 'danceability', 'energy',
       'loudness', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'like', 'predicted like', 'key_0.0',
       'key_1.0', 'key_2.0', 'key_3.0', 'key_4.0', 'key_5.0', 'key_6.0',
       'key_7.0', 'key_8.0', 'key_9.0', 'key_10.0', 'key_11.0', 'mode_0.0',
       'mode_1.0'],
      dtype='object')


I am counting the different genres and the amount of times each one is displayed. I then display a graph with the top 15 genres and an other category. 

In [51]:
# Concatenate datasets
combined_df = pd.concat([df1, df], ignore_index=True)

combined_df.head()
print(combined_df.head())

X = combined_df.drop(['like', 'predicted like'], axis=1)
y = combined_df['like']

# Split the data into training and testing sets with stratification
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

# Convert all columns to numeric
X_train = X_train.apply(pd.to_numeric, errors='coerce')
X_val = X_val.apply(pd.to_numeric, errors='coerce')

# Convert all columns to numeric for X_train and X_val
X_train = X_train.astype(float)
X_val = X_val.astype(float)

# Ensure there are no NaN or infinite values
X_train = np.nan_to_num(X_train, nan=0.0, posinf=1e10, neginf=-1e10)
X_val = np.nan_to_num(X_val, nan=0.0, posinf=1e10, neginf=-1e10)

# Ensure the labels are integers
y_train = y_train.astype(int)
y_val = y_val.astype(int)

                                  trackName               artistName  \
0                               Gamesofluck                  Parcels   
1  So Much In Love - Armin van Buuren Remix  D.O.D, Armin van Buuren   
2                  Envious - Rezident Remix          Aluna, Rezident   
3                          Change Your Mind                 Interupt   
4                                   Isolate                    D.O.D   

                                               genre  danceability  energy  \
0                          aussietronica, indie soul         0.659   0.757   
1  electro house, house, pop dance, uk dance, dut...         0.672   0.769   
2             dance pop, german house, melodic house         0.659   0.710   
3                                           bassline         0.677   0.864   
4          electro house, house, pop dance, uk dance         0.662   0.884   

   loudness  speechiness  acousticness  instrumentalness  liveness  ...  \
0    -5.275       0.038

In [52]:
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model on the validation set
accuracy = model.evaluate(X_val, y_val)
accuracy

Epoch 1/10
228/228 [==============================] - 1s 3ms/step - loss: 0.6569 - accuracy: 0.8985 - val_loss: 0.4437 - val_accuracy: 0.9413
Epoch 2/10
228/228 [==============================] - 0s 2ms/step - loss: 0.3457 - accuracy: 0.9206 - val_loss: 0.1320 - val_accuracy: 0.9413
Epoch 3/10
228/228 [==============================] - 0s 2ms/step - loss: 0.2356 - accuracy: 0.9343 - val_loss: 0.0757 - val_accuracy: 0.9413
Epoch 4/10
228/228 [==============================] - 0s 2ms/step - loss: 0.1102 - accuracy: 0.9541 - val_loss: 0.0513 - val_accuracy: 0.9622
Epoch 5/10
228/228 [==============================] - 0s 2ms/step - loss: 0.0644 - accuracy: 0.9793 - val_loss: 0.0330 - val_accuracy: 0.9978
Epoch 6/10
228/228 [==============================] - 0s 2ms/step - loss: 0.0645 - accuracy: 0.9892 - val_loss: 0.0201 - val_accuracy: 0.9989
Epoch 7/10
228/228 [==============================] - 0s 2ms/step - loss: 0.0643 - accuracy: 0.9941 - val_loss: 0.0130 - val_accuracy: 0.9995
Epoch 

[0.005872142966836691, 0.9994514584541321]

In this cell i am setting up my variable and training with the logisticRegression model. Then I display the shape and initial accuracy. 


testing the accuracy of the model. 